In [7]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model 
from keras.layers import Flatten, Dropout, Dense, GlobalAveragePooling2D, AveragePooling2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from utility import preprocess_image
from keras.models import load_model 
import sys
import os
sys.path.insert(0, 'model')
import tensorflow as tf

In [8]:
#Get Training Config Parameters
run_dir = 'train_run'
nb_train_samples = 18190
nb_validation_samples = 2022
save_step = 10
epochs = 30000
batch_size = 128
learning_rate = 0.1
momentum = 0.99
fine_tuning_ture = 0
fine_tuning_model_path = "./train_run/XXX.hdf5"
initial_epoch=0

# GPU Usage Control
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
K.set_session(sess)

In [9]:
#prepare network
# base model selection
bm = 2
# dimensions of our images.
img_width, img_height = 100,100
num_classes = 452
#0 -- VGG16 (224x224)
#1 -- Mobilenet (224x224)
#2 -- CASIA (100*100)

#Strarting from random initialization
#if fine tuning from pre-trained model : weight = 'imagenet'
if(fine_tuning_ture == 0):
    #define input image size
    if K.image_data_format() == 'channels_first':
        input_shape = (1, img_width, img_height)
    else:
        input_shape = (img_width, img_height, 1)
    
    #load predefined base network model
    if (bm==0):
        from keras.applications.vgg16 import VGG16 #VGG
        base_model = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    elif (bm==1):
        from keras.applications.mobilenet import MobileNet #Mobilenet
        base_model = MobileNet(include_top=False, weights='imagenet', input_shape=input_shape)
    elif (bm==2):
        from casia import CASIA 
        base_model = CASIA(input_shape=input_shape)
    
    x = base_model.output
    
    if bm == 2:
        x = Flatten()(x)
    
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
else:
    model = load_model(fine_tuning_model_path)  

In [10]:
#para has problem
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100, 100, 1)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 100, 100, 32)      320       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 50, 50, 64)        36928     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 25, 25, 128)       0         
__________

In [ ]:
# prepare ImageDataGenerator
train_data_dir = './dataset/lfw_mtcnnpy_160_gray_train_valid/train'
validation_data_dir = './dataset/lfw_mtcnnpy_160_gray_train_valid/valid'
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
#     horizontal_flip=True,
#     fill_mode='nearest',
#     rotation_range=20,
#     width_shift_range=0.9,
#     height_shift_range=0.9,
#     shear_range=0.2,
#     zoom_range=0.2,
    preprocessing_function=preprocess_image)
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode="grayscale",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle = True)
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    color_mode="grayscale",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle = True)

Found 18109 images belonging to 452 classes.
Found 2103 images belonging to 452 classes.


In [ ]:
#start to train
#compile model
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=learning_rate, momentum=momentum), metrics=['accuracy']    )

#setup check point
filepath=os.path.join(run_dir, "weights-improvement-{epoch:02d}-{val_loss:.2f}-{acc:.2f}-{val_acc:.2f}.hdf5")
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
checkpoint = ModelCheckpoint(filepath,period=save_step)
callbacks_list = [checkpoint]

#start to train
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks_list,
    initial_epoch=initial_epoch)

Epoch 1/30000
142/142 [==============================] - 35s 245ms/step - loss: 14.1962 - acc: 0.0096 - val_loss: 15.9922 - val_acc: 0.0078
Epoch 2/30000
142/142 [==============================] - 21s 151ms/step - loss: 16.0418 - acc: 0.0047 - val_loss: 16.0845 - val_acc: 0.0021
Epoch 3/30000
142/142 [==============================] - 20s 138ms/step - loss: 16.0764 - acc: 0.0026 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 4/30000
142/142 [==============================] - 20s 139ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 5/30000
142/142 [==============================] - 20s 139ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 6/30000
142/142 [==============================] - 20s 139ms/step - loss: 16.0763 - acc: 0.0026 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 7/30000
142/142 [==============================] - 20s 138ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 8/30000
142/14

142/142 [==============================] - 32s 227ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 60/30000
142/142 [==============================] - 20s 139ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 61/30000
142/142 [==============================] - 30s 212ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 62/30000
142/142 [==============================] - 20s 138ms/step - loss: 16.0763 - acc: 0.0026 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 63/30000
142/142 [==============================] - 19s 137ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 64/30000
142/142 [==============================] - 31s 217ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 65/30000
142/142 [==============================] - 19s 137ms/step - loss: 16.0763 - acc: 0.0026 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 66/30000
142/142 [====

142/142 [==============================] - 20s 137ms/step - loss: 16.0763 - acc: 0.0026 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 118/30000
142/142 [==============================] - 20s 138ms/step - loss: 16.0754 - acc: 0.0027 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 119/30000
142/142 [==============================] - 20s 138ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 120/30000
142/142 [==============================] - 20s 138ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 121/30000
142/142 [==============================] - 20s 138ms/step - loss: 16.0763 - acc: 0.0026 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 122/30000
142/142 [==============================] - 20s 138ms/step - loss: 16.0763 - acc: 0.0026 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 123/30000
142/142 [==============================] - 20s 139ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 124/30000
142/14

Epoch 175/30000
142/142 [==============================] - 26s 180ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 176/30000
142/142 [==============================] - 20s 138ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 177/30000
142/142 [==============================] - 19s 137ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 178/30000
142/142 [==============================] - 19s 137ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 179/30000
142/142 [==============================] - 20s 138ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 180/30000
142/142 [==============================] - 20s 138ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 181/30000
142/142 [==============================] - 32s 224ms/step - loss: 16.0763 - acc: 0.0026 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 

142/142 [==============================] - 20s 141ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 233/30000
142/142 [==============================] - 20s 139ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 234/30000
142/142 [==============================] - 20s 140ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 235/30000
142/142 [==============================] - 20s 139ms/step - loss: 16.0763 - acc: 0.0026 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 236/30000
142/142 [==============================] - 20s 140ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 237/30000
142/142 [==============================] - 20s 139ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 238/30000
142/142 [==============================] - 20s 140ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 239/30000
142/14

...

142/142 [==============================] - 26s 182ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 348/30000
142/142 [==============================] - 20s 144ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 349/30000
142/142 [==============================] - 21s 145ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 350/30000
142/142 [==============================] - 22s 156ms/step - loss: 16.0763 - acc: 0.0026 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 351/30000
142/142 [==============================] - 21s 148ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 352/30000
142/142 [==============================] - 20s 140ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 353/30000
142/142 [==============================] - 20s 141ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 354/30000
142/14

Epoch 405/30000
142/142 [==============================] - 26s 186ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 406/30000
142/142 [==============================] - 20s 139ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 407/30000
142/142 [==============================] - 20s 140ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 408/30000
142/142 [==============================] - 27s 190ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 409/30000
142/142 [==============================] - 20s 139ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 410/30000
142/142 [==============================] - 28s 195ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 411/30000
142/142 [==============================] - 21s 146ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 

142/142 [==============================] - 35s 247ms/step - loss: 16.0763 - acc: 0.0026 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 463/30000
142/142 [==============================] - 20s 139ms/step - loss: 16.0763 - acc: 0.0026 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 464/30000
142/142 [==============================] - 30s 209ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 465/30000
142/142 [==============================] - 20s 140ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 466/30000
142/142 [==============================] - 20s 140ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 467/30000
142/142 [==============================] - 26s 180ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 468/30000
142/142 [==============================] - 22s 152ms/step - loss: 16.0763 - acc: 0.0026 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 469/30000
142/14

Epoch 520/30000
142/142 [==============================] - 20s 143ms/step - loss: 16.0763 - acc: 0.0026 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 521/30000
142/142 [==============================] - 21s 150ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 522/30000
142/142 [==============================] - 20s 141ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 523/30000
142/142 [==============================] - 21s 144ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 524/30000
142/142 [==============================] - 21s 145ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 525/30000
142/142 [==============================] - 21s 145ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 526/30000
142/142 [==============================] - 21s 145ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 

142/142 [==============================] - 21s 146ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 578/30000
142/142 [==============================] - 21s 146ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 579/30000
142/142 [==============================] - 20s 142ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 580/30000
142/142 [==============================] - 21s 148ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 581/30000
142/142 [==============================] - 21s 150ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 582/30000
142/142 [==============================] - 21s 149ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 583/30000
142/142 [==============================] - 22s 154ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0761 - val_acc: 0.0026
Epoch 584/30000
142/14